# <p style="background-color:lightgreen;font-family:newtimeroman;color:black;font-size:150%;text-align:center;border-radius:50px 50px;">Введение в обработку естественного языка</p>

<h2 align='center'>Урок 5. Part-of-Speech разметка, NER, извлечение отношений</h2>  

<h3 align='left'>Практическое задание:</h3>  

**Задание 1. Написать теггер на данных с русским языком**

    1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
    2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
    3. сравнить все реализованные методы, сделать выводы  

**Задание 2. Проверить, насколько хорошо работает NER**

Данные брать из Index of /pub/named_entities

    1. проверить NER из nltk/spacy/deeppavlov.
    2. написать свой NER, попробовать разные подходы.
        a. передаём в сетку токен и его соседей.
        b. передаём в сетку только токен.
        c. свой вариант.
    3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

### Библиотеки

In [ ]:
!pip install pyconll
!pip install nvidia-tensorrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import pyconll

from collections import Counter

import numpy as np

from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras import activations
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalAveragePooling1D, BatchNormalization, Dropout, LSTM, Activation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Задание 1

### Данные:

In [ ]:
!mkdir dataset_ru

mkdir: cannot create directory ‘dataset_ru’: File exists


данные взяты из 

https://github.com/UniversalDependencies/UD_Russian-SynTagRus

In [ ]:
!wget -O ./dataset_ru/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-test.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu

--2023-04-18 22:44:55--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘./dataset_ru/ru_syntagrus-ud-train-a.conllu’

./dataset_ru/ru_syn 100%[===================>]  38.85M  99.9MB/s    in 0.4s    

2023-04-18 22:44:55 (99.9 MB/s) - ‘./dataset_ru/ru_syntagrus-ud-train-a.conllu’ saved [40736581/40736581]

--2023-04-18 22:44:55--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw

In [ ]:
# собирем тренировочный датасет из частей

full_train = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-c.conllu')

full_train.extend([*full_train_b, *full_train_c])

In [ ]:
full_test = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-test.conllu')

In [ ]:
# full_test2 = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-dev.conllu')

In [ ]:
for sent in full_train[:3]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT

В ADP
приемной NOUN
его PRON
с ADP
утра NOUN
ожидали VERB
посетители NOUN
, PUNCT
- PUNCT
кое-кто PRON
с ADP
важными ADJ
делами NOUN
, PUNCT
а CCONJ
кое-кто PRON
и PART
с ADP
такими DET
, PUNCT
которые PRON
легко ADV
можно ADV
было AUX
решить VERB
в ADP
нижестоящих ADJ
инстанциях NOUN
, PUNCT
не PART
затрудняя VERB
Семена PROPN
Еремеевича PROPN
. PUNCT



In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [ ]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len("" if token.form is None else token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [ ]:
all_train_texts = [' '.join("" if token.form is None else token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join("" if token.form is None else token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join("" if token.upos is None else token.upos for token in sent) for sent in full_train]
all_test_labels = [' '.join("" if token.upos is None else token.upos for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))
print('\n'.join(all_train_labels[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [ ]:
example_tag = fdata_sent_test[0]
example_tag

['В',
 'советский',
 'период',
 'времени',
 'число',
 'ИТ',
 '-',
 'специалистов',
 'в',
 'Армении',
 'составляло',
 'около',
 'десяти',
 'тысяч',
 '.']

### UnigramTagger

In [ ]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(example_tag), unigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'PROPN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', 'VERB'),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.8880238497616922

### BigramTagger

In [ ]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(example_tag), bigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'PROPN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', None),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.7229743842925773

Результат хуже, так как многие биграмы довольно редкие и не встречались в тренировочных данных, поэтому для таких биграм нужно использовать униграмы в качестве backoff:

In [ ]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(example_tag), bigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'PROPN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', 'VERB'),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.89297989100507

### TrigramTagger

In [ ]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(example_tag), trigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', None),
 ('ИТ', None),
 ('-', 'PUNCT'),
 ('специалистов', None),
 ('в', 'ADP'),
 ('Армении', None),
 ('составляло', None),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.4143389729664724

Тут аналогичная ситуация, как и с биграмами, теперь воспользуемся в качестве backoff биграмами, которые в свою очередь имеют backoff униграмы:

In [ ]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(example_tag), trigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'NOUN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', 'VERB'),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.8918848780611308

Результаты чуть хуже, чем у биграм с backoff униграм

### Комбинация тэггеров

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
display(tag.tag(example_tag), tag.evaluate(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'NOUN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', 'VERB'),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.9190956332402889

Результат получился лучше предыдущих, так как оказывается, что даже некоторых слов не было в тренировочных данных, поэтому при добавлении DefaultTagger('NOUN'), который неизвестным словам ставит в соответсвие существительное, так как это предположительно самая частая часть речи. 

### Свой тэггер

Напишем нейронную сеть

Предобработка данных:

In [ ]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    norm_sent = []
    label_sent = []
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        norm_sent.append(tok[0])
        label_sent.append(tok[1])
    train_tok.append(norm_sent)
    train_label.append(label_sent)
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    norm_sent = []
    label_sent = []
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        norm_sent.append(tok[0])
        label_sent.append(tok[1])
    test_tok.append(norm_sent)
    test_label.append(label_sent)

In [ ]:
corpus = [token for sent in train_tok for token in sent]
corpus[:5]

['Анкета', '.', 'Начальник', 'областного', 'управления']

In [ ]:
freq_dict = Counter(corpus)
freq_dict_sorted = sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[(',', 97978),
 ('.', 64114),
 ('в', 30986),
 ('и', 30064),
 ('"', 21639),
 ('-', 19280),
 ('не', 15028),
 ('на', 14637),
 ('что', 10590),
 ('с', 9559)]

In [ ]:
vocabulary_size = len(freq_dict)
vocabulary_size

132847

In [ ]:
VOCABULARY_SIZE = vocabulary_size
VOCABULARY_SIZE

132847

In [ ]:
top_n_words = [word for word, freq in freq_dict_sorted[:VOCABULARY_SIZE]]
top_n_words[:5]

[',', '.', 'в', 'и', '"']

In [ ]:
vocab = {y:x for x,y in enumerate(top_n_words, start=1)}
list(vocab.items())[:5]

[(',', 1), ('.', 2), ('в', 3), ('и', 4), ('"', 5)]

In [ ]:
train_tok_num = [[vocab.get(i, 0) for i in j] for j in train_tok]
test_tok_num = [[vocab.get(i, 0) for i in j] for j in test_tok]
print(train_tok[0])
print(train_tok_num[0])

['Анкета', '.']
[39888, 2]


In [ ]:
max_len = max(len(sent) for sent in train_tok_num)
max_len

205

In [ ]:
MAX_LEN = max_len
MAX_LEN

205

In [ ]:
x_train_nn = [i+[0]*(MAX_LEN-len(i)) for i in train_tok_num]
x_test_nn = [i+[0]*(MAX_LEN-len(i)) for i in test_tok_num]
x_train_nn[0][:10]

[39888, 2, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
x_train_nn_array = np.array([[j for j in i] for i in x_train_nn])
x_test_nn_array = np.array([[j for j in i] for i in x_test_nn])
x_train_nn_array[0]

array([39888,     2,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [ ]:
tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']

In [ ]:
num2tag = dict(zip(range(1, len(tags)+1), tags))
num2tag

{1: 'ADJ',
 2: 'ADP',
 3: 'ADV',
 4: 'AUX',
 5: 'CCONJ',
 6: 'DET',
 7: 'INTJ',
 8: 'NOUN',
 9: 'NUM',
 10: 'PART',
 11: 'PRON',
 12: 'PROPN',
 13: 'PUNCT',
 14: 'SCONJ',
 15: 'SYM',
 16: 'VERB',
 17: 'X'}

In [ ]:
tag2num = {j:i for i,j in num2tag.items()}
tag2num

{'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [ ]:
y_train_nn = [[tag2num[i] for i in j]+[0]*(MAX_LEN-len(j)) for j in train_label]
y_test_nn = [[tag2num[i] for i in j]+[0]*(MAX_LEN-len(j)) for j in test_label]
y_train_nn[0][:10]

[8, 13, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
y_train_nn_array = np.array([[j for j in i] for i in y_train_nn])
y_test_nn_array = np.array([[j for j in i] for i in y_test_nn])
y_train_nn_array[0]

array([ 8, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0])

Модель:

In [ ]:
embedding_dim = 64
hidden_dim = 32
dropout = 0.2

In [ ]:
model = Sequential([
    Embedding(VOCABULARY_SIZE+1, embedding_dim), # VOCABULARY_SIZE+1 так как еще 0, который показывает где нет слова
    BatchNormalization(),
    Dropout(dropout),

    LSTM(embedding_dim, return_sequences=True),
    BatchNormalization(),
    Dropout(dropout),
    Activation(activations.relu),

    Dense(hidden_dim, activation='relu'),
    BatchNormalization(),
    Dropout(dropout),

    Dense(len(tags)+1, activation='softmax') # len(tags)+1 так как еще 0, который показывает где нет тега
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          8502272   
                                                                 
 batch_normalization (BatchN  (None, None, 64)         256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 lstm (LSTM)                 (None, None, 64)          33024     
                                                                 
 batch_normalization_1 (Batc  (None, None, 64)         256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, None, 64)          0

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
with tf.device("GPU:0"):
  hh = model.fit(x_train_nn_array, y_train_nn_array, epochs=40, batch_size=512)

Epoch 1/50
136/136 [==============================] - 47s 263ms/step - loss: 1.1724 - accuracy: 0.8631
Epoch 2/50
136/136 [==============================] - 24s 177ms/step - loss: 0.1859 - accuracy: 0.9729
Epoch 3/50
136/136 [==============================] - 20s 150ms/step - loss: 0.0711 - accuracy: 0.9875
Epoch 4/50
136/136 [==============================] - 17s 124ms/step - loss: 0.0397 - accuracy: 0.9928
Epoch 5/50
136/136 [==============================] - 15s 110ms/step - loss: 0.0271 - accuracy: 0.9948
Epoch 6/50
136/136 [==============================] - 12s 89ms/step - loss: 0.0209 - accuracy: 0.9957
Epoch 7/50
136/136 [==============================] - 9s 68ms/step - loss: 0.0171 - accuracy: 0.9962
Epoch 8/50
136/136 [==============================] - 9s 68ms/step - loss: 0.0146 - accuracy: 0.9966
Epoch 9/50
136/136 [==============================] - 10s 76ms/step - loss: 0.0129 - accuracy: 0.9968
Epoch 10/50
136/136 [==============================] - 8s 58ms/step - loss: 0.0

Здесь вычисляется accuracy с учётом нулей, нас это не устраивает, поэтому вычислим заново, а так же вычислим для test:

In [ ]:
nn_preds_train = model.predict(x_train_nn_array, verbose=1, batch_size=512)
nn_preds_train[0][:2]

136/136 [==============================] - 2s 12ms/step


array([[3.1422367e-24, 7.2197972e-09, 3.7137193e-09, 2.2093010e-09,
        1.0999845e-09, 3.0601652e-12, 8.3016144e-10, 6.5742072e-11,
        1.0000000e+00, 3.0359095e-09, 1.0733720e-14, 3.2602110e-10,
        2.5851785e-08, 4.6526242e-12, 9.3042282e-11, 4.9947393e-11,
        9.9578958e-09, 1.3953514e-09],
       [1.7488636e-25, 3.3843346e-09, 1.6606497e-09, 9.3342660e-11,
        5.1412870e-09, 3.6808126e-10, 4.8723986e-10, 6.9108205e-11,
        1.6890167e-12, 5.6907448e-09, 1.3038866e-09, 1.9617299e-10,
        4.3328285e-08, 9.9999988e-01, 9.9808543e-11, 1.7770946e-11,
        8.6758184e-11, 7.6586147e-08]], dtype=float32)

In [ ]:
acc_train = 0
for i in range(len(x_train_nn_array)):
    len_sent = len(full_train[i])
    acc_train += sum(np.argmax(nn_preds_train[i][:len_sent], axis=1) == y_train_nn_array[i][:len_sent])/len_sent
acc_train/len(x_train_nn_array)

0.9935468810991621

In [ ]:
nn_preds_test = model.predict(x_test_nn_array, verbose=1, batch_size=512)
nn_preds_test[0][:2]

18/18 [==============================] - 0s 16ms/step


array([[5.3670907e-29, 8.7837012e-12, 9.9999905e-01, 4.8864734e-07,
        1.1500502e-08, 1.9924787e-08, 1.7445196e-08, 3.4546281e-07,
        4.9748844e-10, 2.0830697e-10, 2.7181279e-08, 3.8997523e-08,
        3.0565811e-08, 4.5029309e-09, 8.9661015e-13, 3.2679173e-09,
        3.7177270e-09, 6.6532633e-09],
       [3.4395186e-31, 9.9999952e-01, 4.7669364e-12, 1.9406718e-07,
        1.1630180e-10, 3.7352594e-15, 7.9585639e-08, 6.0669791e-13,
        2.1160759e-10, 1.9382108e-08, 1.3440235e-10, 4.1304016e-11,
        7.7728842e-08, 8.1818033e-12, 3.6445194e-10, 1.9032291e-12,
        3.6835102e-09, 2.0490959e-12]], dtype=float32)

In [ ]:
acc_test = 0
for i in range(len(x_test_nn_array)):
    len_sent = len(full_test[i])
    acc_test += sum(np.argmax(nn_preds_test[i][:len_sent], axis=1) == y_test_nn_array[i][:len_sent])/len_sent
acc_test/len(x_test_nn_array)

0.9101200792479872

### Вывод:

Лучше всех отработал комбинированный теггер, нейронка тоже неплохо справилась, но при этом для неё нужно подготовить данные, построить модель, обучить и предсказать, это всё оказалось гораздо дольше, чем работать с теггерами из коробки. К тому же, комбинированный теггер показал метрику даже лучше.

## Задание 2

### Данные:

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2023-04-18 22:54:56--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip.1’

collection5.zip.1   100%[===================>]   1.81M  1.91MB/s    in 0.9s    

2023-04-18 22:54:57 (1.91 MB/s) - ‘collection5.zip.1’ saved [1899530/1899530]



In [ ]:
!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Collection5/001.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace Collection5/002.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install -qq corus

In [ ]:
from corus import load_ne5

path_coll5 = 'Collection5/'
records = load_ne5(path_coll5)
next(records)

Ne5Markup(
    id='339',
    text='Глава МИД Египта М.Ораби ушел в отставку\r\n\r\nВ Египте ушел в отставку глава министерства иностранных дел страны Мухаммед Осман Ораби. Как сообщили представители властей, египетский премьер-министр Эссам Шараф принял отставку руководителя внешнеполитического ведомства, передает Би-Би-Си. Они уточнили, что глава правительства также назначил двух вице-премьеров в лице Хазема Беблави и Али аль-Силми.\r\n\r\nОтметим, что в понедельник, 18 июля, Э.Шараф намерен начать перестановки в правительстве, чтобы успокоить демонстрантов, недовольных медленным ходом реформ в североафриканском государстве.\r\n\r\nМ.Ораби возглавлял внешнеполитическое ведомство Египта всего на протяжении месяца: 19 июня он сменил на посту руководителя МИД Набиля аль-Араби, которым в середине мая 2011г. был избран генеральным секретарем Лиги арабских государств.\r\n\r\nНапомним, новое правительство Египта пришло к власти в стране в результате государственного переворота. Бывший презид

In [ ]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from razdel import tokenize

In [ ]:
words_docs = []
text_data = []
for ix, rec in enumerate(records):
    text_data.append(rec.text)
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [ ]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [ ]:
df_words['tag'].value_counts()

OUT         219017
PER          21172
ORG          13646
LOC           4568
GEOPOLIT      4350
MEDIA         2481
Name: tag, dtype: int64

In [ ]:
df_words.head(3)

,word,tag
0,Медведев,PER
1,уволил,OUT
2,Суходольского,PER


In [ ]:
df_words.shape

(265234, 2)

### NLTK

С помощью функции nltk.ne_chunk () мы можем распознавать именованные сущности с помощью классификатора, который добавляет метки категорий, такие как PERSON, ORGANIZATION и GPE.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
from tqdm import tqdm

In [ ]:
result_nltk = []
for text in tqdm(text_data):
  result_nltk.append({(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label')})

100%|██████████| 999/999 [01:29<00:00, 11.13it/s]


In [ ]:
result_nltk[:2]

[{('Алексей Малых', 'PERSON'),
  ('Валерий Белоцерковский', 'PERSON'),
  ('Денис Иванов', 'PERSON'),
  ('Леонтьев', 'ORGANIZATION'),
  ('Леонтьеву', 'PERSON'),
  ('Малых', 'PERSON'),
  ('Медведев', 'PERSON'),
  ('Михаил Суходольский', 'PERSON'),
  ('Михаила Суходольского', 'PERSON'),
  ('РИА Новости', 'ORGANIZATION'),
  ('Рашида Нургалиева', 'PERSON'),
  ('Рашидом Нургалиевым', 'PERSON'),
  ('Рашиду Нургалиеву', 'PERSON'),
  ('России', 'PERSON'),
  ('России Дмитрий Медведев', 'PERSON'),
  ('СМИ', 'ORGANIZATION'),
  ('Сергей Умнов', 'PERSON'),
  ('Сообщалось', 'PERSON'),
  ('Умнов', 'PERSON')},
 {('Виктор', 'PERSON'),
  ('Виктор Шалиманов', 'PERSON'),
  ('Шалиманов', 'ORGANIZATION')}]

### Spacy

In [ ]:
!pip -qq install spacy
!python -m spacy download en
!python -m spacy download en_core_web_sm

2023-04-18 22:57:28.110398: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-04-18 22:57:48.525607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!pip install -qq -U spacy
!python -m spacy info
import spacy
from spacy import displacy
import en_core_web_sm

2023-04-18 22:58:17.555265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.5.2                         
Location         /usr/local/lib/python3.9/dist-packages/spacy
Platform         Linux-5.10.147+-x86_64-with-glibc2.31
Python version   3.9.16                        
Pipelines        en_core_web_sm (3.5.0)        



In [ ]:
nlp = en_core_web_sm.load()
result_spacy = []
for text in tqdm(text_data):
    result_spacy.append(nlp(text))

100%|██████████| 999/999 [01:00<00:00, 16.55it/s]


In [ ]:
for article in result_spacy[:2]:
    displacy.render(article, jupyter=True, style='ent')

### Deeppavlov

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install deeppavlov

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m deeppavlov install ner_ontonotes_bert_torch

2023-04-18 23:06:57.899 WARNING in 'deeppavlov.core.common.file'['file'] at line 39: 

################################################################################
# The model 'ner_ontonotes_bert_torch' has been removed from the DeepPavlov configs.
# The model 'ner_ontonotes_bert' is used instead.
# To disable this message please switch to 'ner_ontonotes_bert'.
# Automatic name resolving will be disabled in the deeppavlov 1.2.0,
# and if you try to use 'ner_ontonotes_bert_torch' you will get an ERROR.
################################################################################

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
deeppavlov_ner = build_model('ner_ontonotes_bert_torch', download=True)

2023-04-18 23:07:58.426 WARNING in 'deeppavlov.core.common.file'['file'] at line 39: 

################################################################################
# The model 'ner_ontonotes_bert_torch' has been removed from the DeepPavlov configs.
# The model 'ner_ontonotes_bert' is used instead.
# To disable this message please switch to 'ner_ontonotes_bert'.
# Automatic name resolving will be disabled in the deeppavlov 1.2.0,
# and if you try to use 'ner_ontonotes_bert_torch' you will get an ERROR.
################################################################################


################################################################################
# The model 'ner_ontonotes_bert_torch' has been removed from the DeepPavlov configs.
# The model 'ner_ontonotes_bert' is used instead.
# To disable this message please switch to 'ner_ontonotes_bert'.
# Automatic name resolving will be disabled in the deeppavlov 1.2.0,
# and if you try to use 'ner_ontonotes_bert_torch' you w

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Как видно, модель ждёт длину не более 512 символов, поэтому предварительно нужно разделить на предложения.

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
result_deeppavlov = []
for text in tqdm(text_data):
  res = []
  for sent in sent_tokenize(text):
    if len(sent)<=512:
      res.extend(deeppavlov_ner([sent])[1][0])
    else:
      for i in range(1, (-1*len(sent)//512*-1)+1):
        res.extend(deeppavlov_ner([sent[(i-1)*512:i*512]])[1][0])
  result_deeppavlov.append(res)

100%|██████████| 999/999 [04:26<00:00,  3.75it/s]


In [ ]:
result_deeppavlov[:2]

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DATE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DATE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'

### Свой NER

In [ ]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#### a. Передаём в сетку токен и его соседей.

In [ ]:
a_x = []
a_y = []
for i in range(len(words_docs)):
  a_y.append(words_docs[i][1])
  if i==0:
    a_x.append(['', words_docs[i][0], words_docs[i+1][0]])
  elif i==(len(words_docs)-1):
    a_x.append([words_docs[i-1][0], words_docs[i][0], ''])
  else:
    a_x.append([words_docs[i-1][0], words_docs[i][0], words_docs[i+1][0]])

In [ ]:
count_ner = dict(sorted(Counter(a_y).items(), key=lambda x: -x[1]))
ner2num = dict(zip(count_ner.keys(), range(len(count_ner))))
ner2num

{'OUT': 0, 'PER': 1, 'ORG': 2, 'LOC': 3, 'GEOPOLIT': 4, 'MEDIA': 5}

In [ ]:
for i in range(len(a_y)):
    a_y[i]=[ner2num[a_y[i]]]

In [ ]:
bow = []
for i in a_x:
  for j in i:
    bow.append(j)
count_bow = dict(sorted(Counter(bow).items(), key=lambda x: -x[1]))
word2num = dict(zip(count_bow.keys(), range(1, len(count_bow))))
list(word2num.items())[:10]

[(',', 1),
 ('.', 2),
 ('в', 3),
 ('"', 4),
 ('и', 5),
 ('на', 6),
 ('с', 7),
 ('что', 8),
 ('по', 9),
 ('В', 10)]

In [ ]:
vocab_size = len(word2num)
vocab_size

34926

In [ ]:
for i in range(len(a_x)):
  for j in range(3):
    a_x[i][j]=word2num.get(a_x[i][j], 0)

In [ ]:
from sklearn import model_selection, preprocessing, linear_model

train_x, test_x, train_y, test_y = model_selection.train_test_split(a_x, a_y)

In [ ]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size+1, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
mmodel = modelNER()

In [ ]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
mmodel.fit(train_x, train_y, epochs=3)

Epoch 1/3
6217/6217 [==============================] - 45s 7ms/step - loss: 0.3542 - accuracy: 0.8896
Epoch 2/3
6217/6217 [==============================] - 34s 5ms/step - loss: 0.1620 - accuracy: 0.9442
Epoch 3/3
6217/6217 [==============================] - 35s 6ms/step - loss: 0.0994 - accuracy: 0.9644


Вместо передачи слова, также передаём его соседей слева и справа, но предсказываем мы так же только одну метку для слова по центру.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_y, np.argmax(mmodel.predict(test_x), axis=1)))

2073/2073 [==============================] - 4s 2ms/step
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     54782
           1       0.82      0.81      0.82      5181
           2       0.78      0.68      0.73      3508
           3       0.77      0.64      0.70      1137
           4       0.63      0.59      0.61      1062
           5       0.85      0.72      0.78       639

    accuracy                           0.92     66309
   macro avg       0.80      0.74      0.76     66309
weighted avg       0.92      0.92      0.92     66309



Видно, что 0 класс предсказывается лучше всего, а 4 класс предсказывает хуже всего, посмотрим, что это за классы:

In [ ]:
num2ner = {j:i for i,j in ner2num.items()}
print(0, num2ner[0])
print(4, num2ner[4])

0 OUT
4 GEOPOLIT


В целом метрики очень хорошие, теперь посмотрим, что будет, если передовать по одному слову:

#### b. передаём в сетку только токен.

In [ ]:
b_x = []
for i in range(len(words_docs)):
  b_x.append(words_docs[i][0])

In [ ]:
for i in range(len(b_x)):
    b_x[i]=word2num.get(b_x[i], 0)

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(b_x, a_y)

Только уберём MaxPool

In [ ]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size+1, embedding_dim)
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = self.emb(x)
        
        fc_x = self.fc1(x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
mmodel = modelNER()

In [ ]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
mmodel.fit(train_x, train_y, epochs=3)

Epoch 1/3
6217/6217 [==============================] - 34s 5ms/step - loss: 0.2881 - accuracy: 0.9166
Epoch 2/3
6217/6217 [==============================] - 31s 5ms/step - loss: 0.1208 - accuracy: 0.9634
Epoch 3/3
6217/6217 [==============================] - 29s 5ms/step - loss: 0.1078 - accuracy: 0.9656


In [ ]:
print(classification_report(test_y, np.argmax(mmodel.predict(test_x), axis=1)))

2073/2073 [==============================] - 8s 4ms/step
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     54706
           1       0.57      0.83      0.68      5287
           2       0.83      0.54      0.66      3494
           3       0.80      0.82      0.81      1099
           4       0.89      0.88      0.89      1103
           5       0.96      0.75      0.84       620

    accuracy                           0.91     66309
   macro avg       0.84      0.79      0.80     66309
weighted avg       0.92      0.91      0.91     66309



Усреднённые метрики такие же или даже чуть лучше, при этом мы просто обучались на словах, тоесть даже не пытались уловить контекст и всё равно accuracy=0.92.

Стоит отметить, что теперь у 4 класса метрки одни из лучших, это может говорить о том, что геополитику проще распознать по частотным словам, которых там видимо мало и они постоянно повторяются, чем через контекст.

#### c. свой вариант

Я попробую улучшить вариант с соседями (вариант a), только добавлю ещё туда слой lstm, который будет проходить в двух направлениях, чтобы всё таки попробывать распознавать контекст, а не затачиваться под частотные слова

In [ ]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size+1, embedding_dim)
        self.lstm_to = LSTM(embedding_dim, return_sequences=True)
        self.lstm_back = LSTM(embedding_dim, return_sequences=True, go_backwards=True)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = self.emb(x)

        lstm_x = self.lstm_to(x)
        lstm_x = self.lstm_back(x)

        concat_x = tf.concat([x, lstm_x], axis=1)
        pool_x = self.gPool(concat_x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(a_x, a_y)

In [ ]:
mmodel = modelNER()

In [ ]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
mmodel.fit(train_x, train_y, epochs=3)

Epoch 1/3


6217/6217 [==============================] - 56s 8ms/step - loss: 0.2228 - accuracy: 0.9326
Epoch 2/3
6217/6217 [==============================] - 47s 7ms/step - loss: 0.0460 - accuracy: 0.9859
Epoch 3/3
6217/6217 [==============================] - 54s 9ms/step - loss: 0.0254 - accuracy: 0.9920


In [ ]:
print(classification_report(test_y, np.argmax(mmodel.predict(test_x), axis=1)))

2073/2073 [==============================] - 6s 2ms/step
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     54689
           1       0.99      0.92      0.96      5281
           2       0.81      0.88      0.84      3498
           3       0.91      0.83      0.87      1168
           4       0.94      0.88      0.91      1058
           5       0.93      0.86      0.89       615

    accuracy                           0.97     66309
   macro avg       0.93      0.89      0.91     66309
weighted avg       0.97      0.97      0.97     66309



Результаты очень хорошие. Оказывается, если предсказывать по контексту, используя соседей, то метрики будут лучше, просто до этого мы не использовали lstm слой, который как раз и помогает модели понять контекст, а так же я использовал связку из двух слоёв, где первый идёт в прямом направлении, а второй в обратном, таким образом у нас есть информация и по левому и по правому слову, это честно, так как когда человек пытается выделить сущность, то он в том числе смотрит и на следующее слово, так как оно тоже несёт очень много информации.

Так же я почти не настраивал модель и обучал всего 3 эпохи, я почти уверен, что этой архитектурой можно добиться и 0.99-1.00 accuracy, если не будет получаться, то можно попробывать взять большего размера окно, размером не 3, а например 5, тоесть слева будет 2 слова-соседа и справа 2 слова-соседа.

## Задание 3

### Вывод:

Я сравнил подходы к задаче Ner и вывел classification_report для каждого. Я сделал выводы после каждого подхода и убедился, что предсказывать по котнексту, тоесть зная информацию о соседях, в задаче Ner гораздо эффективнее, чего и следовало ожидать.